In [1]:
import numpy as np
import pandas as pd
import matplotlib as plt
import re

In [2]:
dataset = pd.read_csv('E:\ELL_project\problem1\health_data.csv')
dataset = dataset.sample(frac=1)
X = dataset.iloc[:,:-1].values
y = dataset.iloc[:,-1:].values
datasize = X.shape[0]
X_train = X[:(datasize*7)//10,:]
y_train = y[:(datasize*7)//10,:]
X_test = X[(datasize*7)//10:,:]
y_test = y[(datasize*7)//10:,:]

In [1]:
def sep_01(X_train,y_train):
    x_0_train = X_train[y_train[:,0]==0]
    x_1_train = X_train[y_train[:,0]==1]
    return(x_0_train,x_1_train)

In [2]:
def mean_var(x):
    siz = x.shape[0]
    mean = np.sum(x,axis=0)/siz
    var = np.sum(np.square(x-mean),axis=0) / siz
    return(mean,var)

In [ ]:
def hypothesis(x,)